In [1]:
%matplotlib inline
# %load_ext autoreload
# %autoreload 2

import sys; sys.path.insert(0, r'C:\Users\Lukas\Documents\projects\esinet')
import pickle as pkl
import numpy as np
from copy import deepcopy
import mne
import seaborn as sns
import matplotlib.pyplot as plt
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet import forward

plot_params = dict(surface='white', hemi='both', verbose=0)

In [2]:
info = forward.get_info()
info['sfreq'] = 100
fwd = forward.create_forward_model(info=info, fixed_ori=True)
fwd_free = forward.create_forward_model(info=info, fixed_ori=False)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    1.1s remaining:    1.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(

In [52]:
n_samples = 1
duration_of_trial = 100
settings = dict(duration_of_trial=duration_of_trial, method='standard', number_of_sources=10)

sim_lstm = Simulation(fwd, info, verbose=True, settings=settings).simulate(n_samples=n_samples)
sim_lstm.source_data[0].plot(**plot_params)

Simulate Source


  0%|          | 0/1 [00:00<?, ?it/s]

Converting Source Data to mne.SourceEstimate object


  0%|          | 0/1 [00:00<?, ?it/s]


Project sources to EEG...

Create EEG trials with noise...


  0%|          | 0/1 [00:00<?, ?it/s]


Convert EEG matrices to a single instance of mne.Epochs...


In [56]:
import pandas as pd

epochs = sim_lstm.eeg_data.set_eeg_reference('average', projection=True)
epochs.apply_baseline(baseline=(None, None))
evoked = epochs.average()
noise_cov = mne.make_ad_hoc_cov(evoked.info, std=dict(eeg=1), verbose=verbose)
data_cov = mne.compute_covariance(epochs, method='empirical', verbose=verbose)
verbose=0
df = pd.DataFrame()
pick_oris = [None, 'max-power']
regs = [0.05]
reduce_ranks = [True, False]
weight_norms = [None, 'unit-noise-gain']
noise_covs = [None]
list_of_dicts = []
fwds = [fwd, fwd_free]
fwds_description = ["fixed", "free"]
for reg in regs:
    for pick_ori in pick_oris:
        for reduce_rank in reduce_ranks:
            for weight_norm in weight_norms:
                for noise_cov in noise_covs:
                    for i, forward in enumerate(fwds):
                        try:

                            lcmv_filter = mne.beamformer.make_lcmv(evoked.info, forward, data_cov, 
                                reg=reg, noise_cov=None, verbose=verbose, pick_ori=pick_ori,
                                weight_norm=weight_norm,
                                reduce_rank=reduce_rank)
                            
                            stc = mne.beamformer.apply_lcmv(evoked, lcmv_filter, verbose=verbose)
                            predicted_sources = stc.data
                            true_sources = sim_lstm.source_data[0].data
                            corr_score = util.batch_corr(true_sources, predicted_sources)
                            nmse_score = util.batch_nmse(true_sources, predicted_sources)


                            d = dict(noise_cov=noise_cov is not None, reg=reg, pick_ori=pick_ori, reduce_rank=reduce_rank, weight_norm=weight_norm, orientation=fwds_description[i], corr_score=corr_score, nmse_score=nmse_score)
                            list_of_dicts.append(d)
                        except:
                            continue
                
                
                

Applying baseline correction (mode: mean)


C:\Users\Lukas\AppData\Local\Temp/ipykernel_12180/2685781762.py:3: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  epochs = sim_lstm.eeg_data.set_eeg_reference('average', projection=True)


In [57]:
df = pd.DataFrame([list_of_dicts[0]])

for d in list_of_dicts[1:]:
    df = df.append(pd.DataFrame([d]))

df.sort_values('corr_score', ascending=False)
# data_frames = [pd.DataFrame.from_dict(d.items()).T for d in list_of_dicts]
# pd.concat(data_frames, axis=0)

,noise_cov,reg,pick_ori,reduce_rank,weight_norm,corr_score,nmse_score,orientation
0,False,0.05,max-power,False,unit-noise-gain,0.109458,0.133570,free
0,False,0.05,max-power,False,None,0.103348,0.077934,free
0,False,0.05,None,False,None,0.061236,0.062414,fixed
0,False,0.05,None,False,unit-noise-gain,0.053059,0.121548,fixed
0,False,0.05,None,True,unit-noise-gain,0.001233,0.201990,free
0,False,0.05,None,True,None,-0.001536,0.154476,free
0,False,0.05,None,False,None,-0.002485,0.116067,free
0,False,0.05,None,False,unit-noise-gain,-0.003558,0.212525,free
0,False,0.05,max-power,True,None,-0.044561,0.117731,free
0,False,0.05,max-power,True,unit-noise-gain,-0.058160,0.156934,free


In [40]:
epochs = sim_lstm.eeg_data.set_eeg_reference('average', projection=True)
epochs.apply_baseline(baseline=(None, None))
evoked = epochs.average()
verbose=0

# noise_cov = mne.compute_covariance(epochs, method='empirical', verbose=verbose)
noise_cov = mne.make_ad_hoc_cov(evoked.info, std=dict(eeg=1), verbose=verbose)

data_cov = mne.compute_covariance(epochs, method='empirical', verbose=verbose)

lcmv_filter = mne.beamformer.make_lcmv(evoked.info, fwd_free, data_cov, reg=0.01, noise_cov=None, verbose=verbose, pick_ori='max-power',
                         weight_norm=None,
                         reduce_rank=False)
stc = mne.beamformer.apply_lcmv(evoked, lcmv_filter, verbose=verbose)
stc.plot(**plot_params)

Applying baseline correction (mode: mean)


C:\Users\Lukas\AppData\Local\Temp/ipykernel_12180/3200938942.py:1: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  epochs = sim_lstm.eeg_data.set_eeg_reference('average', projection=True)


  File "C:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1348, in _on_button_release
    self.picked_renderer = self.plotter.iren.FindPokedRenderer(x, y)
AttributeError: 'RenderWindowInteractor' object has no attribute 'FindPokedRenderer'


Using control points [0.00012708 0.00014079 0.00018594]


  File "C:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1348, in _on_button_release
    self.picked_renderer = self.plotter.iren.FindPokedRenderer(x, y)
AttributeError: 'RenderWindowInteractor' object has no attribute 'FindPokedRenderer'


Using control points [0.00119012 0.00137611 0.00179529]
Using control points [0.0022437  0.00300996 0.00457056]


  File "C:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1348, in _on_button_release
    self.picked_renderer = self.plotter.iren.FindPokedRenderer(x, y)
AttributeError: 'RenderWindowInteractor' object has no attribute 'FindPokedRenderer'


Using control points [0.00012984 0.00014924 0.00025057]
Using control points [0.0022437  0.00300996 0.00457056]
